# Votes Vs Odds

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('sofascore_data.xlsx')

### Run betting simulation where we bet against voters when discrepancy between votes and odds is large. Calculate profit per bet.

In [ ]:
discrepancies = np.arange(0, 1, 0.1) # Discrepancy levels between probability implied by odds and probability implied by votes
votes_ = np.arange(1000, 10000, 1000) # Options for total number of votes

bets = []

# Loop over all discrepancy levels and number of votes
for discrepancy in discrepancies:
    for votes in votes_:

        # Events that meet the conditions
        high_votes = df[df['total votes'] > votes]
        high_votes = high_votes[(abs(high_votes['votes prob away'] - high_votes['impl prob away'])/high_votes['votes prob away'] > discrepancy) |
                                (abs(high_votes['votes prob home'] - high_votes['impl prob home'])/high_votes['votes prob home'] > discrepancy)]

        # Skip iteration if there are less than 100 events meeting the conditions
        if len(high_votes) < 100:
            continue
        
        balance = 1000

        for id, event in high_votes.iterrows():

            # Bet on home player if votes overestimate chances of away player winning
            if event['votes prob away'] > event['impl prob away']:
                
                bet = {'Event': event['players'],
                      'Sign': 1,
                      'Outcome': event['winner'],
                      'Profit': 1-1 / event['odd home'] if event['winner'] == 1 else -1 / event['odd home'],
                      'Odds': event['odd home'],
                      'votes threshold': votes,
                      'discrepancy': discrepancy
                      }
                bets.append(bet)
                balance += bet['Profit']
                
            # Bet on away player if votes overestimate chances of home player winning
            if event['votes prob away'] < event['impl prob away']:
                
                bet = {'Event': event['players'],
                      'Sign': 2,
                      'Outcome': event['winner'],
                      'Profit': 1-1 / event['odd away'] if event['winner'] == 2 else -1 / event['odd away'],
                      'Odds': event['odd away'],
                      'votes threshold': votes,
                      'discrepancy': discrepancy
                      }
                bets.append(bet)
                balance += bet['Profit']

bets = pd.DataFrame(bets)

### Create graph for profit per bet vs 'votes threshold' and 'discrepancy'

In [ ]:
profit_per_bet = bets.groupby(['votes threshold', 'discrepancy']).apply(lambda x: x['Profit'].sum() / len(x)).reset_index(name='profit_per_bet')

# Pivot table to get grid
pivot = profit_per_bet.pivot(index='discrepancy', columns='votes threshold', values='profit_per_bet')

X = pivot.columns.values
Y = pivot.index.values
X, Y = np.meshgrid(X, Y)
Z = pivot.values

import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(
    z=Z, x=X, y=Y, colorscale='Viridis', 
    contours={"z": {"show": True, "usecolormap": True, "highlightcolor": "limegreen", "project_z": True}}
)])

fig.update_layout(
    title='Profit per Bet by Votes Threshold and Discrepancy',
    scene=dict(
        xaxis_title='Votes Threshold',
        yaxis_title='Discrepancy',
        zaxis_title='Profit per Bet'
    ),
    autosize=True,
    width=900,
    height=700,
    margin=dict(l=50, r=50, b=50, t=50)
)

fig.show()

C:\Users\SamCoenenSYMSON\AppData\Local\Temp\ipykernel_17592\2757529495.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

